In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# importing the csv file, do basic inspection, dropping NaN values

In [2]:
# depression by level of education 2014 dataset
df_edu = pd.read_csv('../data/depression-by-level-of-education-employment.csv')
df_gen = pd.read_csv('../data/gender/gender_hdi.csv')
df_country = pd.read_csv('../data/number-with-depression-by-country.csv')
df_devInd = pd.read_csv('../data/human-development-index.csv')
df_depr = pd.read_csv('../data/prevalence-of-depression-by-age.csv')
df_happiness = pd.read_csv('../data/world_happiness_index.csv')

functions used to check for mismatches in different csv files are the same, in this case country names

In [53]:
def list_diff(list1, list2): 
    return (list(set(list1) - set(list2))) 

load and inspect df_country

In [5]:
df_country.head()

,Entity,Code,Year,Prevalence - Depressive disorders - Sex: Both - Age: All Ages (Number)
0,Afghanistan,AFG,1990,318435.813670
1,Afghanistan,AFG,1991,329044.773956
2,Afghanistan,AFG,1992,382544.572895
3,Afghanistan,AFG,1993,440381.507393
4,Afghanistan,AFG,1994,456916.645489


In [6]:
df_country.rename(columns = {'Prevalence - Depressive disorders - Sex: Both - Age: All Ages (Number)': 'absolute_numbers'}, inplace= True)

In [7]:
df_country_nan = df_country[df_country.isna().any(axis=1)]
entity_list = [i for i in df_country_nan['Entity']]
nan_value_countries = set(entity_list)

Since df_country only have absolute numbers, we wil use df_depr due to the avaibility of the percentage

In [12]:
df_depr.rename(columns ={'Prevalence - Depressive disorders - Sex: Both - Age: Age-standardized (Percent)':'depression_percentage'}, inplace=True)

In [14]:
df_depr = df_depr[['Entity','Code','Year','depression_percentage']]

In [16]:
df_depr.sample(5)

,Entity,Code,Year,depression_percentage
5336,Southern Latin America,NaN,2006,3.796182
584,Belize,BLZ,2014,2.837371
6270,Vietnam,VNM,2016,2.879813
395,Bahamas,BHS,1993,2.648670
1135,"Central Europe, Eastern Europe, and Central Asia",NaN,2005,3.361671


did a lot of work on pivoting the table, just to realise at the end it would be bad practice, so all codes have been turned to comments

In [306]:
# Code practices on pandas.series

In [307]:
# test = df_country.groupby(['Entity']).absolute_numbers.mean()
# test
# type(test)
# indexlist = [i for i in test.index]
# len(indexlist) 
# test.loc['Germany']

In [308]:
# code practices ond pivot_table and multiIndex, here pivot_table is not needed since it would turn the already tidy data into untidy data!

In [309]:
#df_country_pivot = df_country.pivot_table(index=['Entity'], columns=['Year'], values=['absolute_numbers'])
#df_country_pivot.loc[:,('absolute_numbers',1995)]
#df_country_pivot.loc['Germany',('absolute_numbers',1995)]
#df_country_pivot.loc['Germany',('absolute_numbers')]
#df_country_pivot.loc['Germany',('absolute_numbers',(1995,2016))]
#df_country_pivot.columns
#df_devInd.head()
# df_devInd_2index = df_devInd.set_index(['Entity','Year'])
# df_devInd_2index.sort_index(inplace = 'True')
# df_devInd_2index.head(5)
# multiindex, visually beautiful, but not tidy
# df_devInd_2index = df_devInd_2index.reset_index()
# len(df_devInd_2index.index)
# df_devInd_2index.columns
# df_devInd_2index.iloc[0,0]
# if dataframe is not pivoted, or pivoted with dual index (country, year), then concat like this:
# merged_df = pd.merge(df1,df2,left_index=True,right_index=True)

loading and cleaning of dataset 'human development index'

In [17]:
df_devInd.rename(columns={'Human Development Index (UNDP)':'HDI_UNDP'},inplace =True)

In [18]:
df_devInd.shape

(5001, 4)

In [20]:
df_devInd.sample(5)

,Entity,Code,Year,HDI_UNDP
2848,Malta,MLT,2015,0.871
2211,Jamaica,JAM,2004,0.691
1127,Cyprus,CYP,2000,0.802
2579,Libya,LBY,2001,0.731
3247,Nicaragua,NIC,1995,0.524


Creating a dataframe consisting of information on HDI and depression level for a range of years

In [21]:
df_depression_hdi = df_depr.merge(df_devInd, how='inner', left_on=['Entity','Year'], right_on=['Entity','Year'])

In [22]:
df_depression_hdi = df_depression_hdi.drop(['Code_y'],axis=1)

In [23]:
df_depression_hdi = df_depression_hdi.rename(columns={'Code_x':'Code'})

In [24]:
df_depression_hdi.isnull().sum()

Entity                   0
Code                     0
Year                     0
depression_percentage    0
HDI_UNDP                 0
dtype: int64

In [25]:
# dataframe on depression level and Hdi
df_depression_hdi.shape

(4680, 5)

In [86]:
len(set(df_depression_hdi['Entity']))

185

In [28]:
df_depression_hdi.tail(7)

,Entity,Code,Year,depression_percentage,HDI_UNDP
4673,Zimbabwe,ZWE,2011,3.109526,0.478
4674,Zimbabwe,ZWE,2012,3.118072,0.505
4675,Zimbabwe,ZWE,2013,3.128192,0.516
4676,Zimbabwe,ZWE,2014,3.140290,0.525
4677,Zimbabwe,ZWE,2015,3.155710,0.529
4678,Zimbabwe,ZWE,2016,3.174134,0.532
4679,Zimbabwe,ZWE,2017,3.192789,0.535


In [88]:
df_depression_hdi.corr(method='pearson')['depression_percentage'].sort_values()

Year                    -0.057807
HDI_UNDP                -0.032316
depression_percentage    1.000000
Name: depression_percentage, dtype: float64

check happiness for nan value and select relevant data for further analysis 

In [51]:
df_happiness.isna().sum()<100

Country name          True
Year                  True
Life Ladder           True
Log GDP per capita    True
dtype: bool

In [63]:
df_happiness = df_happiness[['Country name', 'Year', 'Life Ladder', 'Log GDP per capita']]
df_happiness.rename(columns={'Country name':'Entity','Life Ladder':'life_ladder','Log GDP per capita':'log_gdp_capita'}, inplace=True)

merge happiness index and depression/hdi dataframes

In [64]:
df_hdi_happiness = df_depression_hdi.merge(df_happiness,on=['Entity', 'Year'])

In [78]:
df_hdi_happiness.isnull().sum()

Entity                   0
Code                     0
Year                     0
depression_percentage    0
HDI_UNDP                 0
life_ladder              0
log_gdp_capita           3
dtype: int64

In [81]:
df_hdi_happiness[df_hdi_happiness.isnull().any(axis=1)]

,Entity,Code,Year,depression_percentage,HDI_UNDP,life_ladder,log_gdp_capita
1220,South Sudan,SSD,2017,3.699147,0.388,"2,816622496",NaN
1276,Syria,SYR,2015,3.547811,0.538,"3,46191287",NaN
1467,Yemen,YEM,2017,4.012346,0.452,"3,253560066",NaN


In [87]:
len(set(df_hdi_happiness['Entity']))

155

In [73]:
df_hdi_happiness.sample(5)

,Entity,Code,Year,depression_percentage,HDI_UNDP,life_ladder,log_gdp_capita
37,Argentina,ARG,2013,3.651044,0.820,"6,582260132","9,877256393"
972,Niger,NER,2008,3.437595,0.303,"4,235657215","6,704703808"
861,Mexico,MEX,2009,2.766796,0.743,"6,962819099","9,628098488"
1467,Yemen,YEM,2017,4.012346,0.452,"3,253560066",NaN
571,India,IND,2011,3.520174,0.591,"4,634871483","8,441580772"


load education dataset

In [35]:
df_edu.head(1)

,Entity,Code,All levels of education (active),All levels of education (employed),All levels of education (total),Below upper secondary education (active),Below upper secondary education (employed),Below upper secondary education (total),Tertiary education (active),Tertiary education (employed),Tertiary education (total),Upper secondary and post-secondary non-tertiary education (active),Upper secondary and post-secondary non-tertiary education (employed),Upper secondary and post-secondary non-tertiary education (total)
0,Austria,AUT,6.5,4.7,7.7,15.5,9.0,15.2,4.3,3.5,5.5,5.5,4.2,6.7


In [36]:
df_edu = df_edu[['Entity', 'Code', 'All levels of education (total)', 'Below upper secondary education (total)', 'Tertiary education (total)', 'Upper secondary and post-secondary non-tertiary education (total)']]

In [52]:
# replace slovak republic with slovakis so there the country naming are identical between different dataframes
df_edu = df_edu.replace('Slovak Republic', 'Slovakis')

Creating a Dataframe on education level and depression, dataset on education level is only avaiable for 2014

In [44]:
# select only values for the year 2014
df_depression_hdi_2014 = df_depression_hdi.loc[df_depression_hdi['Year']==2014]
df_depression_hdi_2014

,Entity,Code,Year,depression_percentage,HDI_UNDP
12,Afghanistan,AFG,2014,4.133526,0.491
40,Albania,ALB,2014,2.203684,0.773
68,Algeria,DZA,2014,3.629490,0.747
86,Andorra,AND,2014,3.728379,0.853
105,Angola,AGO,2014,4.156468,0.564
...,...,...,...,...,...
4564,Venezuela,VEN,2014,2.810428,0.778
4592,Vietnam,VNM,2014,2.882767,0.678
4620,Yemen,YEM,2014,4.018785,0.505
4648,Zambia,ZMB,2014,3.552409,0.580


In [46]:
df_depression_hdi_edu = df_edu.merge(df_depression_hdi_2014, left_on='Entity', right_on='Entity')
df_depression_hdi_edu
df_depression_hdi_edu.rename(columns={'All levels of education (total)':'all_levels', 'Below upper secondary education (total)':'below_secondary',
                                     'Tertiary education (total)':'tertiary','Upper secondary and post-secondary non-tertiary education (total)':'secondary_&_above'},
                                      inplace = True)

In [47]:
# create dataframe on the relationship between education level, depression level and hdi for year 2014, since education level is only 
# available for 2014 
df_depression_hdi_edu

,Entity,Code_x,all_levels,below_secondary,tertiary,secondary_&_above,Code_y,Year,depression_percentage,HDI_UNDP
0,Austria,AUT,7.7,15.2,5.5,6.7,AUT,2014,3.267460,0.901
1,Belgium,BEL,7.1,11.6,4.2,7.5,BEL,2014,4.110472,0.909
2,Czech Republic,CZE,4.0,6.0,2.0,4.4,CZE,2014,2.690526,0.879
3,Denmark,DNK,8.3,15.5,6.7,8.8,DNK,2014,3.277973,0.928
4,Estonia,EST,5.1,6.4,4.3,5.2,EST,2014,3.902099,0.864
5,Finland,FIN,10.7,14.4,8.9,13.2,FIN,2014,4.888971,0.914
6,France,FRA,6.3,9.4,4.9,5.7,FRA,2014,4.316753,0.894
7,Germany,DEU,11.6,17.0,8.9,12.5,DEU,2014,3.940119,0.930
8,Greece,GRC,3.6,5.4,2.4,3.4,GRC,2014,4.252704,0.864
9,Hungary,HUN,4.7,10.3,2.8,4.2,HUN,2014,2.753656,0.833


# Bonus: 1. Gender Equality and Depression, 2. Icecream consumption and Depression

In [347]:
df_hdi_gen = df_depression_hdi.merge(df_gen, left_on = 'Entity', right_on = 'Country').drop(['Entity'], axis=1)

In [348]:
df_hdi_gen

,Code,Year,positive_percentage,HDI_UNDP,HDI Rank (2018),Country,1990,1991,1992,1993,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,AFG,2002,4.124928,0.373,170,Afghanistan,0.298,0.304,0.312,0.308,...,0.447,0.464,0.465,0.479,0.485,0.488,0.490,0.491,0.493,0.496
1,AFG,2003,4.125230,0.383,170,Afghanistan,0.298,0.304,0.312,0.308,...,0.447,0.464,0.465,0.479,0.485,0.488,0.490,0.491,0.493,0.496
2,AFG,2004,4.126384,0.398,170,Afghanistan,0.298,0.304,0.312,0.308,...,0.447,0.464,0.465,0.479,0.485,0.488,0.490,0.491,0.493,0.496
3,AFG,2005,4.126908,0.408,170,Afghanistan,0.298,0.304,0.312,0.308,...,0.447,0.464,0.465,0.479,0.485,0.488,0.490,0.491,0.493,0.496
4,AFG,2006,4.128638,0.417,170,Afghanistan,0.298,0.304,0.312,0.308,...,0.447,0.464,0.465,0.479,0.485,0.488,0.490,0.491,0.493,0.496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171,ZWE,2013,3.128192,0.516,150,Zimbabwe,0.498,0.500,0.485,0.480,...,0.448,0.472,0.490,0.516,0.527,0.537,0.544,0.549,0.553,0.563
4172,ZWE,2014,3.140290,0.525,150,Zimbabwe,0.498,0.500,0.485,0.480,...,0.448,0.472,0.490,0.516,0.527,0.537,0.544,0.549,0.553,0.563
4173,ZWE,2015,3.155710,0.529,150,Zimbabwe,0.498,0.500,0.485,0.480,...,0.448,0.472,0.490,0.516,0.527,0.537,0.544,0.549,0.553,0.563
4174,ZWE,2016,3.174134,0.532,150,Zimbabwe,0.498,0.500,0.485,0.480,...,0.448,0.472,0.490,0.516,0.527,0.537,0.544,0.549,0.553,0.563
